In [ ]:
!pip install contractions

In [ ]:
import pandas
import nltk, re, contractions, spacy
from string import punctuation
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

In [ ]:
lemma = nltk.wordnet.WordNetLemmatizer()
punctuation += '’'
NER = spacy.load("en_core_web_sm")

In [ ]:
def fix(text:str):
  try:
    text = contractions.fix(text)
    # content = content.split(" ")
    patt = '[0-9]+[a-z|A-Z]'
    for word in re.findall(patt, text):
      text = text.replace(word, word[:-1] + " " + word[-1])
    # for word in content:
    #   if re.match(patt, word):
    #     print(word)
  except:
    print(type(text))
    
  return text

In [ ]:
data = pandas.read_csv('./drive/MyDrive/FakeNewsDataset.csv')
for col in list(data.columns):
  print(col, len(data.loc[data[col].isnull()]))

subject 0
date 0
label 0
content 0


In [ ]:
words, tags, word_freq, doc_freq = [], [], [], []

In [ ]:
for row in range(len(data)):
  content = data.loc[row, 'content']
  text = set([])
  NEs = NER(content).ents
  for chunk in NEs:
    if chunk.text not in words:
      words.append(chunk.text)
      tags.append(chunk.label_)
      word_freq.append(NEs.count(chunk))
    else:
      ind = words.index(chunk.text)
      word_freq[ind] += NEs.count(chunk)
    
    content = content.replace(chunk.text, '')
    text.add(chunk.text)
  
  content = re.sub('[%s]'%re.escape(punctuation), '', content).lower()
  content = nltk.word_tokenize(content)
  content = [lemma.lemmatize(word) for word in content]
  tagged = nltk.pos_tag(content)

  for word, tag in tagged:
    if word not in words:
      words.append(word)
      tags.append(tag)
      word_freq.append(tagged.count((word, tag)))
    else:
      ind = words.index(word)
      word_freq[ind] += tagged.count((word, tag))
    
    text.add(word)
  
  while len(doc_freq) < len(words):
    doc_freq.append(0)
  for word in text:
    ind = words.index(word)
    doc_freq[ind] += 1

In [ ]:
dct = {
    'word' : words,
    'tag' : tags,
    'frequency' : word_freq,
    'doc_frequency' : doc_freq
}
dfWords = pandas.DataFrame(dct)

In [ ]:
for col in dfWords.columns:
  print(col, len(dfWords.loc[dfWords[col].isnull()]))

word 0
tag 0
frequency 0
doc_frequency 0


In [ ]:
dfWords.to_csv("./drive/MyDrive/words.csv", index=False)

In [ ]:
import pickle
with open("./drive/MyDrive/word_statis.pkl", "wb") as file:
  pickle.dump(dct, file)

In [ ]:
import pandas

In [ ]:
data = pandas.read_csv("./drive/MyDrive/word_statis.csv")
data = data.drop(data.columns[0], axis=1)

In [ ]:
data.sort_values(["doc_frequency", "frequency"], ascending=[0, 0]).head(20)

,Unnamed: 0,word,tag,frequency,doc_frequency
252,252,said,VBD,882057,31818
98,98,would,MD,249821,23051
72,72,president,NN,238961,22377
7687,7687,Reuters,ORG,28502,21600
108,108,people,NNS,156905,19130
25,25,Trump,PRODUCT,100424,18961
60,60,one,CD,29981,18184
100,100,also,RB,71947,18023
0,0,Donald Trump,PERSON,23378,17353
115,115,say,VBP,64074,17338


In [ ]:
data['tf-idf'] = data['frequency'] / data['doc_frequency']

In [ ]:
data.sort_values(['tf-idf'], ascending=[0]).head(30)

In [ ]:
data.loc[data['doc_frequency']==1]

,Unnamed: 0,word,tag,frequency,doc_frequency,tf-idf
2,2,Eve Message,WORK_OF_ART,1,1,1.0
5,5,Angry Pants,PERSON,1,1,1.0
19,19,Alan Sandoval,PERSON,1,1,1.0
21,21,Marlene,PERSON,1,1,1.0
23,23,Koren,PERSON,1,1,1.0
...,...,...,...,...,...,...
338384,338384,$411 million,MONEY,1,1,1.0
338385,338385,suhkoi,NN,1,1,1.0
338386,338386,perusahaan,NN,1,1,1.0
338387,338387,perdangangan,NN,1,1,1.0


In [ ]:
!pip install deepface

     |████████████████████████████████| 63 kB 1.2 MB/s 
     |████████████████████████████████| 2.3 MB 8.6 MB/s 
     |████████████████████████████████| 462 kB 33.6 MB/s 


In [ ]:
data.columns

Index(['title', 'text', 'subject', 'date'], dtype='object')

In [ ]:
model = DeepFace.build_model("ArcFace")

arcface_weights.h5  will be downloaded to  /root/.deepface/weights/arcface_weights.h5


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/arcface_weights.h5
To: /root/.deepface/weights/arcface_weights.h5
100%|██████████| 137M/137M [00:32<00:00, 4.21MB/s]


In [ ]:
model.summary()

Model: "ResNet34"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 112, 112, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 114, 114, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  1728        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [ ]:
data = pandas.read_csv("./drive/MyDrive/word_statis.csv")

In [ ]:
import numpy

In [ ]:
data["tf-idf"] = numpy.float64(data["frequency"] / data['doc_frequency'])

In [ ]:
data.sort_values(["tf-idf"], ascending=[1])

,word,tag,frequency,doc_frequency,tf-idf
191342,Middle EastThe State Department,ORG,2,2,1.000000
250818,pictwittercomibgado1,NN,2,2,1.000000
250817,yesterdaytimes,RB,2,2,1.000000
250816,magazinethe,NN,2,2,1.000000
250815,classed,VBN,14,14,1.000000
...,...,...,...,...,...
104243,hatesparty,JJ,123,1,123.000000
299041,reervation,NN,196,1,196.000000
19357,newtimes,RB,226,1,226.000000
62478,commuted,VBD,4626,17,272.117647
